# Process <a class="anchor" id="top"></a>

- **Prepared by:**
    
    - [Joy Zhang](https://qsdsan.readthedocs.io/en/latest/authors/Joy_Zhang.html)
    
- **Covered topics:**

    - [1. Process](#s1)
    - [2. Processes](#s2)
    
- **Video demo:**

    - [LastName FirstName](link)
    
To run tutorials in your browser, go to this [Binder page](https://mybinder.org/v2/gh/QSD-Group/QSDsan/main?filepath=%2Fdocs%2Fsource%2Ftutorials).
    
You can also watch a video demo on [YouTube](link) (subscriptions & likes appreciated!).

---
### Note
This tutorial is not ready yet, will come soon!

---

In [ ]:
import qsdsan as qs
print(f'This tutorial was made with qsdsan v{qs.__version__}.')

## 1. Process <a class="anchor" id="s1"></a>

[Back to top](#top)

## 2. Processes <a class="anchor" id="#s2"></a>

[Back to top](#top)